In [13]:
import requests
import os
import zipfile
import geopandas as gpd
import pandas as pd

In [2]:
url_basica="https://catalogov2.sig.cdmx.gob.mx/descargas/csv_shapes_catastro/shapefiles/catastro2021_"
fin=".zip"

In [3]:
#crear directorio para guardar los archivos
os.makedirs("catastro_abierto",exist_ok=True)

In [28]:
#Listado de alcaldías
alcaldias=["AlVARO_OBREGON","AZCAPOTZALCO","BENITO_JUAREZ","COYOACAN","CUAJIMALPA","CUAUHTEMOC","GUSTAVO_A_MADERO","IZTACALCO","IZTAPALAPA","MAGDALENA_CONTRERAS","MIGUEL_HIDALGO","MILPA_ALTA","TLAHUAC","TLALPAN","VENUSTIANO_CARRANZA","XOCHIMILCO"]
#Alcaldías lower
alcaldias_low=[alcaldia.lower() for alcaldia in alcaldias]

In [29]:
#Descarga de archivos csv
url_basica="https://catalogov2.sig.cdmx.gob.mx/descargas/csv_shapes/"
fin=".csv"
for alcaldia in alcaldias_low:
    url=url_basica+alcaldia+fin
    r=requests.get(url)
    with open("catastro_abierto/"+alcaldia+".csv","wb") as f:
        f.write(r.content)
        

In [36]:
#Pegar los archivos csv
df=pd.DataFrame()
for alcaldia in alcaldias_low:
    df_temp=pd.read_csv("catastro_abierto/"+alcaldia+".csv",encoding="latin1")
    df=pd.concat([df,df_temp])
df=df[["liga_ciuda","longitud","latitud"]]
#Extraer cuenta catastral utilizando el texto después de "cuenta_catastral=" y hasta el siguiente "&"
df["cuenta_catastral"]=df["liga_ciuda"].str.extract(r"cuentaCatastral=(.*?)&")
#Eliminar guiones de cuenta catastral
df["cuenta_catastral"]=df["cuenta_catastral"].str.replace("_","")
#Crear una variable para obtener el largo de la cuenta catastral
df["largo"]=df["cuenta_catastral"].str.len()
df

,liga_ciuda,longitud,latitud,cuenta_catastral,largo
0,http://ciudadmx.cdmx.gob.mx:8080/seduvi/fichas...,-99.195565,19.403121,03710701,8
1,http://ciudadmx.cdmx.gob.mx:8080/seduvi/fichas...,-99.194779,19.403212,03710901,8
2,http://ciudadmx.cdmx.gob.mx:8080/seduvi/fichas...,-99.192933,19.401788,03712109,8
3,http://ciudadmx.cdmx.gob.mx:8080/seduvi/fichas...,-99.193317,19.400919,03712204,8
4,http://ciudadmx.cdmx.gob.mx:8080/seduvi/fichas...,-99.193027,19.400621,03712316,8
...,...,...,...,...,...
77650,http://ciudadmx.cdmx.gob.mx:8080/seduvi/fichas...,-99.103666,19.215674,77266524,8
77651,http://ciudadmx.cdmx.gob.mx:8080/seduvi/fichas...,-99.096768,19.230653,77266907,8
77652,http://ciudadmx.cdmx.gob.mx:8080/seduvi/fichas...,-99.093015,19.233633,77267108,8
77653,http://ciudadmx.cdmx.gob.mx:8080/seduvi/fichas...,-99.107928,19.199792,77268427,8


In [37]:
df["largo"].unique()

array([8], dtype=int64)

In [38]:
#Salvar cuentas
df.to_csv("cuentas_catastrales_abiertas.csv",index=False)